# Create an Azure Cosmos DB database bulit to scale
Databases need to scale to meet both the data velocity and volume demands created by global audiences.

Db should automatically to account for new products, and have high throughput and low latency to kee pace with customers expectations.
## Learning objectives
In this module, you will:

- Create an Azure Cosmos DB account
- Set the initial throughput volume for your database using request units
- Choose a partition key
- Create a database and container for NoSQL data in Azure Cosmos DB

Main ideas to remember
- Provisioning Throughput for containers and databases - shared across all containers in the db. Need to estimate number of operations you'll have to support at different times.
- Request unit (RU) measured per second - must be provisioned in advance. Based on approx cost of single GET request on a 1-KB document using doc ID. GET is cheap, Create, replace or delete is more expensive.
- [Find the resquest unit charge in Azure Cosmos DB](https://docs.microsoft.com/en-us/azure/cosmos-db/find-request-unit-charge?tabs=dotnetv2)
- Request unit considerations
  - Item size
  - Item indexing
  - Item property count
  - Indexed properties (limit by policy)
  - Data consistency
  - Query patterns
    - Number of query results
    - Number of predicates
    - Nature of predicates
    - number of user defined functions
    - size of the source data
    - size of the result set
    - Projections
  - script usage
- When you create and account you can provision a min of 400 RU/s, or a maximum of 250,000 RU/s

## Partition strategy
ensures that your database can grow to efficiently run queries and transations. Enables horizontal scaling - adding more partitions as needed. A partition key defines this, **can't be changed**, so deciding on this beforehand is hugely important.

When additional physical partitions are needed, Cosmos DB automatically creates them by splitting exiting ones. No downtime or performance impact for this.

Storage space for data associated with each partition key can't exceed 20 GB< size of one physical partition in Azure Cosmos DB.

So if  individual record going ot be larger than 20 GB< think about using composite key (ie instead of UserID, do userID-date - new partition for each day user visited site)

## Best practices for choosing partition key
- Don't be afraid of choosing partition key that has a large number of values. More values = more scalability
- For read heavy workload, review top 3 to 5 queries you plan on using. Value most frequently included in WHERE clause is a good candidate for partition key.
- For write-heavy workloads, you'll need to understand transactional needs of your workload, because partition key is the scope of multi-document transactions.

For each Cosmos DB, specify partition key that satisfies
- high cardinatlity, this option allows data to distribute evenly across all physical partitions
- Evenly distribute requests. REmember the total number of RU/s is evenly divided across all physical partitions
- Evenly distribute storage. Each partition can grow to 20 gb.

`note Aug 9 2020`: Exercise gives option to create Cosmos account and db using
1. C#
2. Java
3. Azure CLI

Using Azure CLI first time just to save time to try to get as much of DP 200/201 done before tomorrow when I start my training., next time will try C#. NEvermind I actually had made it with the portal first, and using CLI is pretty straightforward, you use the following commands

### Cosmos Account

`az cosmosdb create \
    --name $NAME \
    --kind GlobalDocumentDB \
    --resource-group learn-193db41a-9465-4eeb-9eea-f5306bf499dd`
    
### Cosmos SQL DB

`az cosmosdb sql database create \
    --account-name $NAME \
    --name "Products" \
    --resource-group learn-193db41a-9465-4eeb-9eea-f5306bf499dd`
    
### Container + partition key

`az cosmosdb sql container create \
    --account-name $NAME \
    --database-name "Products" \
    --name "Clothing" \
    --partition-key-path "/productId" \
    --throughput 1000 \
    --resource-group learn-193db41a-9465-4eeb-9eea-f5306bf499dd`